# Сравнение качества моделей
### Содержание

* [Подготовка данных](#1)
* [Сравнение моделей](#2)
    * [Теоретические основы](#21)


## Подготовка данных <a id="1"></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("cleaned_dataset.csv", index_col=0)

Вначале нужно определить, на каких данных будет обучаться модель, а на каких тестироваться. **Традиционный подход** - это разделение исходного набора данных на 3 части (обучение, валидация и тестирование) с пропорции 60/20/20. В данном случае обучающая выборка используется для обучения модели, а валидация и тестирование для получения значения метрики без эффекта переобучения.

Однако существует и другой подход к разбиению данных - разделение на 2 части (обучение и тестирование) по правилу 80-20 (80% тренировочный, 20% тестовый). Зачастую данный метод применяется в тех случаях, когда отсутствует достаточное количество данных как в обучающем, так и в проверочном наборе.  

Перед тем как начать разбивать данные необходимо выделить из исходного набора данных целевую переменную (столбец `DRK_YN`) и сохранить её в отдельную переменную. Ниже приведён код разделения:

In [3]:
x = df.drop(columns=["DRK_YN"], axis=1)
y = df["DRK_YN"]

В ходе выполнения данной работы будет использован подход с разделением исходной выборки на 2 части с пропорцией 80-20, поскольку данный способ является самым популярным способом разбиения данных. Для того, чтобы разбить данные таким образом, существует специальный метод `train_test_split` в библиотеке `scikit-learn`.

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

## Сравнение моделей <a id="2"></a>

### Теоретические основы <a id="21"></a>

После того, как модели были обучены, их необходимо оценить при помощи имеющихся метрик оценки. Всего для задач классификации существует множество оценок. Но все они строятся на основе **матрицы ошибок** (*Confusion Matrix*).

Confusion matrix состоит из 4 основных исходов бинарной классификации: *истинно положительных* (**TP**), *ложно положительных* (**FP**), *истнно отрицательных* (**TN**) и *ложно отрицательных* (**FN**). 

Пусть какой-то набор медицинских данных характерен для данного диагноза. Если наша модель верно определила и поставила положительный класс, тогда это *истинно положительный* исход, если же модель ставит отрицательную метку класса, тогда это *ложно отрицательный* исход. В случае отсутствия диагноза у рассматриваемого набора данных исходы модели остаются аналогичными. Тогда, если модель относит запись к классу положительному, то мы говорим о *ложно положительном* исходе (здоровому человеку ошибочно поставлен диагноз "болен"), и наоборот, если модель определяет запись как отрицательный класс, то от *истинно отрицательный* исход (больному человеку поставлен диагноз "болен").

Одной из наиболее простых метрик, построенных на основе матрицы ошибок является **полнота** (*Recall*). Полнота определяет долю верных прогнозов среди всех данных с положительным классом. Она определяется отношением истинно положительных результатов к общему количеству положительных результатов. Таким образом формула выглядит следующим образом:
$$ Recall = \frac{TP}{TP+FN} $$
В исходной задачи необходимо уделить данной задаче некоторое внимание, поскольку в задаче ошибка нераспознания положительного класса высока.

Однако одна полнота не способна в полной мере оценить точность классификации. Поэтому вместе с ней применяют и другую метрику - **точность** (*Precision*), которая измеряет, насколько хорошо модель справляется с ложно положительными результатами. Более формально, точность принимает в расчет только те точки данных, которые были помечены как положительные, и определяет, сколько среди них истинно положительные. Формула для подсчета данного значения выглядит следующим образом:
$$Precision = \frac{TP}{TP+FP}$$
Данная оценка также является важной, поскольку в рамках данной задачи "цена" ложно положительного класса также высока.

Далее, поскольку ранее было выяснено, что метрики **полнота** и **точность** имеют равную высокую значимость, то лучше всего использовать **F-меру** (*F-score*), которая является средним гармоническим между первой и второй метриками. Почему при этом используется именно среднее гармоническое, а не среднее арифметическое? Потому что если одна метрика очень большая, например 100%, а другая наоборот крайне низкая (50%), то среднее арифметическое будет равняться 75%, что является приемлемым результатом. Однако в реальность такой классификатор имеет крайне низкую точность классификации. Таким образом, формула для подсчета выглядит так:
$$F_1 = \frac{2 \cdot precision \cdot recall}{precision+recall}$$
Если обе метрики большие, то и итоговый результат также высокий. Цель F1-меры состоит в том, чтобы определить, является ли высокой как полнота, так и точность.

Однако иногда требуется, чтобы либо полнота, либо точность превалировала одна над другой. Таким образом, может потребоваться при вычислении F-меры придать одной из них больший вес. Как раз для этого и была придумана $F_\beta$-мера, формула которой представлена ниже:
$$F_\beta = \frac{(1+\beta^2) \cdot precision \cdot recall}{\beta^2 \cdot precision + recall}$$
В данной формуле параметр $\beta$ может принимать любое положительное значение и обозначает уровень значимости одного из критериев. Чем ниже $\beta$, тем более весомой становится точность, а чем выше $\beta$, тем важнее становится полнота.

Из ранее рассмотренных метрик становится очевидно, что существует два типа ошибок: ложно положительные и ложно отрицательные. И существует метрика, которая основана на двух данных ошибках - **кривая рабочей характеристики приемника** (*ROC*). Данная кривая строится на основе двух показателей: **избирательности** (*False Positive Rate (FPR)*) и **полноте** (*True Positive Rate (TPR)*). Данные показатели высчитываются по следующим формулам:
$$ FPR = \frac{FP}{FP + TN} $$
$$ TPR = \frac{TP}{TP + TN} $$
И далее, на основе данных двух метрик строится ROC-кривая, где по горизонтали откладывается FPR, а по вертикали - TPR. Однако сама по себе ROC-кривая может лишь графически показать, какая модель лучше справляется с классификацией. Для того, чтобы сравнить две модели численно, необходимо найти **площадь под ROC-кривой** (*Area Under Curve ROC (AUC ROC)*). Диапазон значений AUC ROC равен $[0.5, 1]$, где чем ближе к 1, тем лучше модель для классификации данных.

### Практическая реализация <a id="22"></a>

Перед началом оценки всех моделей создадим функцию для оценки модели. На вход данной функции подаётся обученная модель классификатора и тестовый набор данных (`x_test` и `y_test`). Данная функция в качестве результата выведет на экран матрицу неточностей классификатора (*Confusion Matrix*) и ROC-кривую, а под рисунками будут показаны результаты основных метрик оценивания. Код разработанной функции представлен ниже:

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

def evaluate_model(model, x_test, y_test, model_name: str) -> None:
    pred = model.predict(x_test)
    plt.subplot(1, 2, 1)
    disp = ConfusionMatrixDisplay.from_estimator(model, x_test, y_test, cmap=plt.cm.Blues)
    disp.ax_.set_title(f'Confusion matrix of {model_name}')

    plt.subplot(1, 2, 2)
    disp = RocCurveDisplay.from_estimator(model, x_test, y_test)
    disp.ax_.set_title(f'ROC Curve of {model_name}')

    plt.show()
    vals = f'''Оценивание модели {model_name} по параметрам:
    Полнота (Recall): {recall_score(y_test, pred):.4f},
    Точность (Precision): {precision_score(y_test, pred):.4f},
    F1-мера (F1-score): {f1_score(y_test, pred):.4f},
    Площадь под ROC-кривой (AUC ROC): {roc_auc_score(y_test, pred):.4f}'''
    print(vals)

Далее создадим классификационные модели на основе полученных ранее оптимальных параметров при кросс-валидации. Сперва импортируем все необходимые компоненты:

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
#from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Далее создадим функцию, благодаря которой будет создан классификатор на основе введённого в неё алгоритма. На вход данная функция будет получать алгоритм классификации с введёнными в него параметрами, а на выходе будет получаться полностью обученные классификатор. Программный код разработанной функции представлен ниже:

In [7]:
def create_model(alg, alg_name, x_train, y_train):
    print("Обучение модели...")
    # Создаем стандартизатор
    standardizer = StandardScaler()

    # Создаём конвейер
    pipe = Pipeline([("standardizer", standardizer), (alg_name, alg)])

    clf = pipe.fit(x_train, y_train)
    print("Обучение завершено!")
    return clf


Далее определим словарь, в котором ключем будет являться название метода классификации, а значением - сам метод классификации.

In [8]:
# Для стака моделей
estimators = [
    ('forest', RandomForestClassifier(n_jobs=-1, n_estimators=793, random_state=42, max_depth=14, max_features=12, min_samples_leaf=17)),
    ('lr', LogisticRegression(solver='saga', n_jobs=-1, C=1.6, l1_ratio=0.60, penalty="elasticnet"))
]

algs = {
    'k-nearest neighbors': KNeighborsClassifier(n_jobs=-1, metric='manhattan', n_neighbors=14),
    'support vector machine': LinearSVC(max_iter=10000, C=1.5, loss="squared_hinge", tol=0.005001),
    'logistic regression': LogisticRegression(solver="saga", n_jobs=-1, C=4.972235551967773, l1_ratio=0.00387986251009187, penalty='elasticnet'),
    'decision tree': DecisionTreeClassifier(min_samples_leaf=50, criterion='log_loss', max_depth=10, max_features=23),
    'random forest': RandomForestClassifier(n_jobs=-1, n_estimators=793, max_depth=12, max_features=12, min_samples_leaf=17),
    'adaptive boost': AdaBoostClassifier(learning_rate=0.5, n_estimators=159),
    #'gradiend boost': XGBClassifier(random_state=42, colsample_bylevel=0.9452050650024018, colsample_bynode=1.0, colsample_bytree=1.0, gamma=5.966638612389175,
    #                                learning_rate=0.08893704430415869, max_depth=17, reg_alpha=20.0, reg_lambda=0.0, subsample=1.0),
    'stacked model': StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(min_samples_leaf=10)),
}

Далее в цикле пройдемся по всем элементам данного словаря и сперва создадим классификатор, а затем оценим его точность.

In [9]:
for name, alg in algs.items():
    print(f'Работа с {name}')
    clf = create_model(alg, name, x_train, y_train)
    evaluate_model(clf, x_test, y_test, name)

Работа с k-nearest neighbors
Обучение модели...
